### Leer el archivo JSON usando el "DataFrameReader" de Spark

### Ingestion de la carpeta "production_company"

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
production_companies_schema = StructType(fields=[
    StructField("companyId", IntegerType(), True),
    StructField("companyName", StringType(), True),

])

In [0]:
productions_companies_df = spark.read.schema(production_companies_schema).csv(
    f"{bronze_folder_path}/{v_file_date}/production_company/"
)

In [0]:
from pyspark.sql.functions import current_timestamp, lit


In [0]:
productions_companies_final_df = productions_companies_df.withColumnsRenamed({
    'companyId': 'company_id',
    'companyName': 'company_name',
}).withColumns({
    'ingestion_date': current_timestamp(),
    'enviroment': lit("production"),
    'file_date': lit(v_file_date)
})

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.productions_companies")

overwrite_partition(productions_companies_final_df, "movie_silver", "productions_companies", "file_date")


In [0]:
#productions_companies_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/productions_companies")

#productions_companies_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.productions_companies")

productions_companies_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.productions_companies")

In [0]:
%fs
ls /mnt/datamoviehistory/silver/movies_casts

In [0]:
%sql
--select file_date, count(1) from movie_silver.productions_companies_final_df group by file_date